In [32]:
import pandas as pd
import numpy as np

%matplotlib inline
pd.set_option("display.max_columns", None)

In [33]:
dftrain = pd.read_csv("../train.csv")

In [34]:
def eda(dataframe):
    temp_maxrows = pd.get_option("display.max_rows")
    pd.set_option("display.max_rows", None)
    print("missing values \n", dataframe.isnull().sum())
    print("dataframe index \n", dataframe.index)
    print("dataframe types \n", dataframe.dtypes)
    print("dataframe shape \n", dataframe.shape)
    print("dataframe describe \n", dataframe.describe())
    print("dataframe duplicates \n", dataframe.duplicated().sum()) 
    for item in dataframe:
        print(item)
        print(dataframe[item].nunique())
    pd.set_option("display.max_rows", temp_maxrows)
        
eda(dftrain)

missing values 
 Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF       

In [35]:
def eda_helper(df):
    temp_maxrows = pd.get_option("display.max_rows")
    pd.set_option("display.max_rows", None)
    dict_list = []
    for col in df.columns:
        data = df[col]
        dict_ = {}
        dict_.update({"null_count" : data.isnull().sum()})
        dict_.update({"unique_count" : len(data.unique())})
        dict_.update({"data_type" : set([type(d).__name__ for d in data])})
        dict_list.append(dict_)
    eda_df = pd.DataFrame(dict_list)
    eda_df.index = df.columns
    eda_df = eda_df.sort_values(['null_count','unique_count'], ascending=[True, False])
    return eda_df
    pd.set_option("display.max_rows", temp_maxrows)

eda_helper(dftrain)

,null_count,unique_count,data_type
Id,0,1460,{int}
LotArea,0,1073,{int}
GrLivArea,0,861,{int}
BsmtUnfSF,0,780,{int}
1stFlrSF,0,753,{int}
TotalBsmtSF,0,721,{int}
SalePrice,0,663,{int}
BsmtFinSF1,0,637,{int}
GarageArea,0,441,{int}
2ndFlrSF,0,417,{int}


In [36]:
#Dropping columns with too many missing or null values
dftrain = dftrain.drop(['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'Utilities'], axis=1)

In [37]:
"""    FEATURE CREATION    """

### MSZoning ###
#Creating a feature 'residential_den' to quantitatively record high, medium and low density.
#Potential introduction of error since we are assuming a relation/preference of low density to high.
dftrain.MSZoning.replace(['RH','RM','RL', 'RP', 'FV'], [2, 1, 0, 0, 0], inplace=True)

#Dropping the few entries in the df that are non-residential properties
index_todrop = dftrain.loc[dftrain.MSZoning == 'C (all)'].index


### Street ###
#This variable is very monolithic. Identifying the 4 entries for Grvl to remove from the df, and dropping the variable
index_todrop = index_todrop.append(dftrain.loc[dftrain.Street == 'Grvl'].index)


### LotShape ###
### LandContour ###
### Neighborhood : Physical lcoations within Ames city limits ###
### Condition1 : Proximity to various conditions ###
### BldgType : Type of dwelling ###
### HouseStyle : Style of dwelling ###
### RoofStyle : Type of roof ###
    #If model needs some bias, this is a potential drop.
### Exterior1st : Exterior covering on house ###
### Exterior2nd : Exterior covering on house (if more than one material) ###
### MasVnrType : Masonry veneer type ###
### Foundation : Type of foundation ###
### MoSold : Month Sold (MM) ###
#Produce and append dummies to convert hard-to-compare entries into logistical features
LotShapedumms = pd.get_dummies(dftrain.LotShape, prefix='LotShape')
LandContourdumms = pd.get_dummies(dftrain.LandContour, prefix='LandContour')
Neighborhooddumms = pd.get_dummies(dftrain.Neighborhood, prefix='nghood')
Condition1dumms = pd.get_dummies(dftrain.Condition1, prefix='con1')
BldgTypedumms = pd.get_dummies(dftrain.BldgType, prefix='BldgType')
HouseStyledumms = pd.get_dummies(dftrain.HouseStyle, prefix='HouseStyle')
RoofStyledumms = pd.get_dummies(dftrain.RoofStyle, prefix='RfStyle')
Ext1dumms = pd.get_dummies(dftrain.Exterior1st, prefix='Ext1')
Ext2dumms = pd.get_dummies(dftrain.Exterior2nd, prefix='Ext2')
MasVnrTypedumms = pd.get_dummies(dftrain.MasVnrType, prefix='MasVnrType')
Foundationdumms = pd.get_dummies(dftrain.Foundation, prefix='Found')
MoSolddumms = pd.get_dummies(dftrain.MoSold, prefix='MoSold')


### LotConfig ###
#Combine the entries for 'Frontage on 2' and '3 sides of the property' together as there are too few FR3 and the value of the data can be preserved as a 'multi frontage' property.
dftrain.LotConfig = dftrain.LotConfig.replace('FR3', 'FR2')
LotConfigdumms = pd.get_dummies(dftrain.LotConfig, prefix='LotConfig')


### LandSlope ###
#Encode the entries in a quantitative way, higher value = more slope. While potentially introducing error, it preserves most of its relationship
dftrain.LandSlope.replace(['Gtl','Mod','Sev'], [0, 1, 2], inplace=True)


### ExterQual : Evaluates the quality of the material on the exterior ###
### ExterCond : Evaluates the present condition of the material on the exterior ###
### BsmtQual : Evaluates the height of the basement ###
### BsmtCond : Evaluates the general condition of the basement ###
### Heating : Type of Heating ###
### HeatingQC : Heating quality and condition ###
### KitchenQual : Kitchen Quality ###
### FireplaceQu : Fireplace quality ###
### GarageQual : Garage quality ###
### GarageCond : Garage condition ###
#Coding the ranked qualitative response into a quantitative, aka A,B,C become 3,2,1
scale_from_6 = sorted(list(range(6)), reverse=True)
rating_from_top = ['Ex','Gd','TA','Fa','Po', np.nan]

dftrain.ExterQual.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.ExterCond.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.BsmtQual.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.BsmtCond.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.Heating.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.HeatingQC.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.KitchenQual.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.FireplaceQu.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.GarageQual.replace(rating_from_top, scale_from_6, inplace=True)
dftrain.GarageCond.replace(rating_from_top, scale_from_6, inplace=True)


### BsmtExposure : Refers to walkout or garden level walls ###
#Coding rating to numeric
dftrain.BsmtExposure.replace(['Gd','Av','Mn','No', np.nan], [4, 3, 2, 1, 0], inplace=True)


### BsmtFinType1 : Rating of basement finished area ###
#Assigning NBas for No Basement, to the current np.nan's. There's no conflict since it'll be used for dummies and dropped as k - 1 control
dftrain.BsmtFinType1.replace(np.nan, 'NBas', inplace=True)
BsmtFindumms = pd.get_dummies(dftrain.BsmtFinType1, prefix='BsmtFin')


### CentrailAir : Central air conditioning ###
#Converting to logistical response
dftrain.CentralAir.replace(['Y','N'], [1,0], inplace=True)


### Electrical : Electrical system ###
#Coding quality of electrical system on a scale. Potential introduction of error. Consider using dummies instead
dftrain.Electrical.replace(['SBrkr','FuseA','FuseF','FuseP','Mix'], [4, 3, 2, 1, 0], inplace=True)


### Functional : Home functionality (Assume typical unless deductions are warranted)  ###
#Coding to numeric scale, with 0 being normal and increasing as functionality worsens
dftrain.Functional.replace(['Sal', 'Sev', 'Maj2', 'Maj1','Mod','Min2','Min1','Typ'], 
                           [7, 6, 5, 4, 3, 2, 1, 0], inplace=True)


### GarageType : Garage location ###
#Assiging categorical value to 'no garage', and turning to dummies. 
dftrain.GarageType.replace(np.nan, 'NoGar', inplace=True)
GarageTypedumms = pd.get_dummies(dftrain.GarageType, prefix='Garage')


### GarageYrBlt : Year garage was built ###
#Dropping because of the pontential error from date data and np.nans. Value from garages is recorded elsewhere
dftrain.drop('GarageYrBlt', axis=1, inplace=True)


### GarageFinish : Interior finish of the garage ###
#Assiging categorical value to 'no garage', and turning to dummies. 
dftrain.GarageFinish.replace(np.nan, 'NoGar', inplace=True)
GarageFinishdumms = pd.get_dummies(dftrain.GarageFinish, prefix='GarFin')


### YrSold : Year Sold (YYYY) ###
#Creating a feature that tracks the age of the property by subtracting the year built from year sold.
dftrain['Prop_Age'] =  dftrain.YrSold - dftrain.YearBuilt


""" CLEAN UP """

### PavedDrive : Paved driveway # 3SsnPorch : Three season porch area in square feet #
# ScreenPorch : Screen porch area in suare feet # PoolArea  : Pool area in square feet #
# MiscVal : $Value of miscellaneous feature # SaleType : Type of Sale # 
#SaleCondition : Condition of Sale # LowQualFinSF : Low quality finished square feet (all floors) #
# BsmtFinType2 : Rating of basement finished area (if multiple types) # 
#BsmtUnfSF : Type 2 finished square feet # Heating : Type of Heating # RoofMatl : Roof material #
# Condition2 : Proximity to various conditions (if more than one is present) ###
#Monolithic responses, especially considering the few choices. Dropping
dftrain.drop(['PavedDrive', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
              'SaleType', 'SaleCondition', 'LowQualFinSF', 'BsmtFinType2', 'BsmtUnfSF',
             'Heating', 'RoofMatl', 'Condition2'], axis=1, inplace=True)                                                        
        
#Dropping all the raw variables that were converted to features
dftrain.drop(['MSSubClass', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'Neighborhood',
             'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 
              'MasVnrType', 'Foundation', 'BsmtFinType1', 'GarageType', 'GarageFinish',
              'MoSold', 'YearBuilt', 'YrSold'], axis=1, inplace=True)

#Concatenating all dummies into DataFrame
dftrain = pd.concat([dftrain, LotShapedumms, LandContourdumms, LotConfigdumms, Neighborhooddumms, 
                    Condition1dumms, BldgTypedumms, HouseStyledumms, RoofStyledumms, Ext1dumms, 
                     Ext2dumms, MasVnrTypedumms, Foundationdumms, BsmtFindumms, GarageTypedumms,
                    GarageFinishdumms, MoSolddumms], axis=1)

#Dropping one variable from every dummy created in accordance with the k - 1 principle. 
#Dropping highest response dummy, if no common sense alternative exists (i.e. one of the responses can be interpreted as a zero)
dftrain.drop(['LotShape_Reg', 'LandContour_Lvl', 'LotConfig_Inside', 'nghood_Blueste', 'con1_Norm',
              'BldgType_1Fam', 'HouseStyle_1Story', 'RfStyle_Gable', 'Ext1_VinylSd', 'Ext2_VinylSd',
             'MasVnrType_None', 'Found_PConc', 'BsmtFin_NBas', 'Garage_NoGar', 'GarFin_NoGar', 
             'MoSold_1'], axis=1, inplace=True)

#Dropping irrelevant (commercial) properties and resetting the index
dftrain.drop(index_todrop, inplace=True)
dftrain.reset_index(drop=True, inplace=True)

In [ ]:
#Get the data ready for modeling
y = dftrain.SalePrice
X = dftrain.drop(['Id', 'SalePrice'], axis=1)

In [ ]:
y.head()

In [38]:
dftrain.tail()


,Id,MSZoning,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,SalePrice,Prop_Age,LotShape_IR1,LotShape_IR2,LotShape_IR3,LandContour_Bnk,LandContour_HLS,LandContour_Low,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,nghood_Blmngtn,nghood_BrDale,nghood_BrkSide,nghood_ClearCr,nghood_CollgCr,nghood_Crawfor,nghood_Edwards,nghood_Gilbert,nghood_IDOTRR,nghood_MeadowV,nghood_Mitchel,nghood_NAmes,nghood_NPkVill,nghood_NWAmes,nghood_NoRidge,nghood_NridgHt,nghood_OldTown,nghood_SWISU,nghood_Sawyer,nghood_SawyerW,nghood_Somerst,nghood_StoneBr,nghood_Timber,nghood_Veenker,con1_Artery,con1_Feedr,con1_PosA,con1_PosN,con1_RRAe,con1_RRAn,con1_RRNe,con1_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RfStyle_Flat,RfStyle_Gambrel,RfStyle_Hip,RfStyle_Mansard,RfStyle_Shed,Ext1_AsbShng,Ext1_AsphShn,Ext1_BrkComm,Ext1_BrkFace,Ext1_CBlock,Ext1_CemntBd,Ext1_HdBoard,Ext1_ImStucc,Ext1_MetalSd,Ext1_Plywood,Ext1_Stone,Ext1_Stucco,Ext1_Wd Sdng,Ext1_WdShing,Ext2_AsbShng,Ext2_AsphShn,Ext2_Brk Cmn,Ext2_BrkFace,Ext2_CBlock,Ext2_CmentBd,Ext2_HdBoard,Ext2_ImStucc,Ext2_MetalSd,Ext2_Other,Ext2_Plywood,Ext2_Stone,Ext2_Stucco,Ext2_Wd Sdng,Ext2_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_Stone,Found_BrkTil,Found_CBlock,Found_Slab,Found_Stone,Found_Wood,BsmtFin_ALQ,BsmtFin_BLQ,BsmtFin_GLQ,BsmtFin_LwQ,BsmtFin_Rec,BsmtFin_Unf,Garage_2Types,Garage_Attchd,Garage_Basment,Garage_BuiltIn,Garage_CarPort,Garage_Detchd,GarFin_Fin,GarFin_RFn,GarFin_Unf,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12
1441,1456,0,62.0,7917,0,6,5,2000,0.0,3,3,4,3,1,0,0,953,5,1,4.0,953,694,1647,0,0,2,1,3,1,3,7,0,1,3,2,460,3,3,0,40,0,175000,-8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1442,1457,0,85.0,13175,0,6,6,1988,119.0,3,3,4,3,1,790,163,1542,3,1,4.0,2073,0,2073,1,0,2,0,3,1,3,7,1,2,3,2,500,3,3,349,0,0,210000,-32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
1443,1458,0,66.0,9042,0,7,9,2006,0.0,5,4,3,4,1,275,0,1152,5,1,4.0,1188,1152,2340,0,0,2,0,4,1,4,9,0,2,4,1,252,3,3,0,60,0,266500,-69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1444,1459,0,68.0,9717,0,5,6,1996,0.0,3,3,3,3,2,49,1029,1078,4,1,3.0,1078,0,1078,1,0,1,0,2,1,4,5,0,0,0,1,240,3,3,366,0,112,142125,-60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
1445,1460,0,75.0,9937,0,5,6,1965,0.0,4,3,3,3,1,830,290,1256,4,1,4.0,1256,0,1256,1,0,1,1,3,1,3,6,0,0,0,1,276,3,3,736,68,0,147500,-43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
